[Accueil](../../../index.ipynb) > [Sommaire de Terminale](../../index.ipynb)

# Recherche textuelle : l'algorithme de Boyer Moore

La recherche d'un **motif** (pattern en anglais) dans une chaîne de caractère a de nombreuses applications concrètes :

- **Moteurs de recherche** : Dans les moteurs de recherche simples ou les outils de recherche locaux, Boyer-Moore peut être utilisé pour trouver des occurrences de mots-clés dans de grands volumes de texte.
- **Éditeurs de texte** : Les éditeurs de texte (Notepad++, Sublime Text, ...) utilisent des algorithmes comme Boyer-Moore pour implémenter la fonctionnalité de recherche rapide de mots ou de phrases.
- **Bioinformatique** : Recherche de séquences d'ADN ou alignement de séquences;
- **Détection de virus** : Les logiciels antivirus utilisent des algorithmes de recherche de motifs pour identifier des signatures de virus ou de malware dans des fichiers ou des paquets réseau;
- **Correcteurs orthographiques**
- **Interception de communications** (Echelon)
- ...

Comment savoir si une chaîne de caractère possède tel **motif**?

Essayons tout d'abord avec le premier algorithme qui vient à l'esprit (algorithme naïf).

## Algorithme naïf

Le motif _roseraie_ est-il présent dans la chaîne de caractère suivante : _Cueillez dès aujourd’hui les roses de la vie_ ?

Une approche naïve est la suivante:

|C|u|e|i|l|l|e|z| |d|è|s| |a|u|j|o|u|r|d|’|h|u|i| |l|e|s| |r|o|s|e|s| |d|e| |l|a| |v|i|e|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
|r|o|s|e|r|a|i|e| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |

C et r sont différents on décale alors le mot _roseraie_

|C|u|e|i|l|l|e|z| |d|è|s| |a|u|j|o|u|r|d|’|h|u|i| |l|e|s| |r|o|s|e|s| |d|e| |l|a| |v|i|e|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
| |r|o|s|e|r|a|i|e| | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | | |

u et r ne correspondent pas,

on décale jusqu'à obtenir la première correspondance
 
|C|u|e|i|l|l|e|z| |d|è|s| |a|u|j|o|u|r|d|’|h|u|i| |l|e|s| |r|o|s|e|s| |d|e| |l|a| |v|i|e|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
| | | | | | | | | | | | | | | | | | |**r**|o|s|e|r|a|i|e| | | | | | | | | | | | | | | | | | |

ici il y a bien correspondance entre _r_ et _r_, il faut donc comparer la lettre suivante. _d_ et _o_ ne correspondent pas, on continue le décalage...

|C|u|e|i|l|l|e|z| |d|è|s| |a|u|j|o|u|r|d|’|h|u|i| |l|e|s| |r|o|s|e|s| |d|e| |l|a| |v|i|e|
|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|-|
| | | | | | | | | | | | | | | | | | | | | | | | | | | | | |**r**|**o**|**s**|**e**|r|a|i|e| | | | | | | |

ici c'est le _s_ et le _r_ qui ne correspondent pas.

### Implémentation de l'algorithme naïf

Voici le pseudo code de l'Algorithme 

```
naive_search(text, pattern)
    mettre dans t la longueur de text
    mettre dans p la longueur de pattern
    initialiser le résultat à une liste vide
    Pour i = 0 à t − p (inclu)
        j <- 0;
        Tant que (j < p et text[j+i] = pattern[j])
            j := j + 1
        Fin Tant que
        Si i = j
            ajouter i dans le résultat
        Fin de Si
    Fin Pour
    Retourner le résultat
```

**Implémenter** la fonction *naive_search*, dans un fichier *horspool.py* qui prend en paramètre :
- le texte *text*;
- le motif *pattern*.

et qui retourne
- la liste des index de *text* pour lesquels il y a correspondance;

**Ajouter** le paramètre nommé case_sensitive qui vaut True par défaut. Si sa valeur est passée à False, la recherche ne doit pas tenir compte de la casse.

Par exemple:
- naive_search("ATACGAGGGGGGGGGGGGGGGGGATAC", "ATAC") doit retourner [0, 23]
- naive_search("AAAAAAAAAAAAAAAAAAAAAAAAAAA", "TGTC") retourne []

**Tester la fonction**

Voici des tests à vérifier:

Ajouter un fichier *test_horspool.py* et ajouter ces tests:

```python
from horspool import naive_search

#   +---------------------------------------------------+
#   |                     TESTS                         |
#   +---------------------------------------------------+

DATAS_SEARCH = (
    # TEXT          PATTERN         CASE_SENSITIVE  EXPECTED
    ("text",        "texto",        True,           []),
    ("text",        "text",         True,           [0]),
    ("Text",        "text",         True,           []),
    ("le text",     "text",         True,           [3]),
    ("xt text text","text",         True,           [3,8]),
    ("text text te","text",         True,           [0,5]),
    ("TexT tEXt te","tExt",         False,          [0,5]),
)

#   ------NAIVE_SEARCH-------------
for text, pattern, case_sensitive, expected in DATAS_SEARCH:
    result = naive_search(text, pattern, case_sensitive=case_sensitive)
    assert expected==result, f"La recherche naïve de '{pattern}' dans '{text}' aurait du être {expected} mais c'est {result}."

```

### Coût de cet algorithme.

On appelle $M$ la longeur du motif et $T$ la longeur du texte.

Dans le pire des cas on doit décaler $(T-M)$ fois, puis faire jusqu'à $M$ comparaisons.
On obtient donc un coût $(T-M)xM$ que l'on approximera à **$TxM$** car souvent $T>>M$.

Ainsi :

- plus le texte est grand plus le coût augmente, ce qui est normal
- plus le motif est long, plus le coût augmente.

Nous allons étudier maintentant un algorithme beaucoup plus efficace.

## Algorithme de Boyer-Moore

Pourquoi faire un algorithme plus rapide?

A cause de la taille des données à traiter : par exemple en bioinformatique, la recherche de [transcriptome](https://fr.wikipedia.org/wiki/Transcriptome) (plusieurs Mo) dans le génome humain (plusieurs Go).

### Un peu d'histoire

Cet algorithme a été développé en 1977 par :

<figure style="float:left; margin:10px 50px 10px 0px">
    <img src="img/rsb.jpg"
         alt="Robert S. Boyer"
         title="Robert S. Boyer"
         style="border:1px solid black;height:200px"
         >
    <figcaption><a href="https://en.wikipedia.org/wiki/Robert_S._Boyer">Robert S. Boyer</a> (1946-)</figcaption>
</figure>

<figure style="float:left; margin:10px 50px 10px 0px">
    <img src="img/jsm.jpg"
         alt="J Strother Moore"
         title="J Strother Moore"
         style="border:1px solid black;height:200px"
         >
    <figcaption><a href="https://en.wikipedia.org/wiki/J_Strother_Moore">J Strother Moore</a></figcaption>
</figure>

<div style="clear:both" class="alert alert-info">
    Nous allons utiliser une version simplifiée de l'algorithme de Boyer-Moore. Il s'agit de l'algorithme de Horspool publié en 1980.
</div>

## Principe sur un exemple

Nous allons tout d'abord expliquer le principe de l'algorithme sur un exemple de séquence d'ADN.

L'ADN est composé d'une séquence de 4 bases azotées:

- Adénine (A)
- Cytosine (C)
- Guanine (G)
- Thymine (T)

Soit la séquence suivante : _CAATGTCTGCACCAAGAGGCCGGCAGGT_

Le motif _CGGCAG_ est-il présent dans la chaîne ?

On détermine d'abord la position minimale de chaque lettre (sauf la dernière) du motif en partant par la droite du motif.

Ce qui donne:

|C|G|A|
|-|-|-|
|2|3|1|

**Dans cet algorithme on regarde tout d'abord la dernière lettre du motif, puis on remonte vers la gauche.**

|i     |0 |1 |2 |3 |4 |5         |6 |7 |8 |9 |10|11|12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |-         |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |~~**T**~~ |C |T |G |C |A |C |C |A |A |G |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |C |G |G |C |A |~~**G**~~ |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> NON
  - La lettre _T_ est-elle présente dans notre motif -> NON
    - On saute donc de la longeur du motif (saut maximal)

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11        |12|13|14|15|16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |-         |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |~~**C**~~ |C |A |A |G |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |C |G |G |C |A |~~**G**~~ |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
  

- Y a t-il correspondance ? -> NON
  - La lettre _C_ est-elle présente dans notre motif -> OUI
    - On saute donc de 2 pour les faire correspondre

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11                                  |12|13        |14|15|16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |-                                   |- |-         |- |- |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |**<span style="color:red">C</span>**|C |~~**A**~~ |A |G |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |C |G |G |**<span style="color:red">C</span>**|A |~~**G**~~ |  |  |  |  |  |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> NON
  - La lettre _A_ est-elle présente dans notre motif -> OUI
    - On saute donc de 1 pour les faire correspondre

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13                                   |14        |15|16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |-                                    |-         |- |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |**<span style="color:red">A</span>** |~~**A**~~ |G |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |C |G |G |C |**<span style="color:red">A</span>** |~~**G**~~ |  |  |  |  |  |  |  |  |  |  |  |  |  |


- Y a t-il correspondance ? -> NON
  - La lettre _A_ est-elle présente dans notre motif -> OUI
    - On saute donc de 1 pour les faire correspondre

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13    |14                                   |15    |16|17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |-     |-                                    |-     |- |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |~~A~~ |**<span style="color:red">A</span>** |**G** |A |G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |C |G |G |~~C~~ |**<span style="color:red">A</span>** |**G** |  |  |  |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> OUI
  - On décale sur la gauche
  - Y a t-il correspondance ? -> OUI
    - On décale sur la gauche
    - Y a t-il correspondance ? -> NON
      - La lettre _A_ est-elle présente dans notre motif -> OUI
        - On saute de (1-2 deux décalages)<0 donc on saute de 1...

<div class="alert alert-info">On voit ici les limites de l'algorithme simplifié : le décalage de 1 ne fait pas correspondre les lettres.</div>

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15|16                                      |17|18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-                                       |- |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |G |**~~<span style="color:red">A</span>~~**|G |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |C |G |G |C |A |**~~<span style="color:red">G</span>~~**|  |  |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> NON
  - La lettre _A_ est-elle présente dans notre motif -> OUI
    - On saute donc de 1 pour les faire correspondre

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15    |16    |17                                   |18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-     |-     |-                                    |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |~~G~~ |**A** |**<span style="color:red">G</span>** |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |  |C |G |G |~~C~~ |**A** |**<span style="color:red">G</span>** |  |  |  |  |  |  |  |  |  |  |


- Y a t-il correspondance ? -> OUI
  - On décale sur la gauche
  - Y a t-il correspondance ? -> OUI
    - On décale sur la gauche
    - Y a t-il correspondance ? -> NON
      - La lettre _G_ est-elle présente dans notre motif -> OUI
        - On saute de (3-2=**1**)  (G en position 3, mais on était décalé de 2)

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15                                   |16|17    |18|19|20|21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-                                    |- |-     |- |- |- |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |**<span style="color:red">G</span>** |A |~~G~~ |G |C |C |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |  |  |C |G |**<span style="color:red">G</span>** |C |~~A~~ |G |  |  |  |  |  |  |  |  |  |

- Y a t-il correspondance ? -> OUI
  - On décale sur la gauche
  - Y a t-il correspondance ? -> NON
    - La lettre _G_ est-elle présente dans notre motif -> OUI
      - On saute de (3-1=**2**)  (G position 3, mais on était décalé de 1)

|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15|16|17                                   |18|19|20        |21|22|23|24|25|26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-                                    |- |- |-         |- |- |- |- |- |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |G |A |**<span style="color:red">G</span>** |G |C |**~~C~~** |G |G |C |A |G |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |C |G |**<span style="color:red">G</span>** |C |A |**~~G~~** |  |  |  |  |  |  |  |

etc... etc...


|i     |0 |1 |2 |3 |4 |5 |6 |7 |8 |9 |10|11|12|13|14|15|16|17|18|19|20    |21    |22    |23    |24    |25    |26|27|
|-     |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |- |-     |-     |-     |-     |-     |-     |- |- |
|chaîne|C |A |A |T |G |T |C |T |G |C |A |C |C |A |A |G |A |G |G |C |**C** |**G** |**G** |**C** |**A** |**G** |G |T |
|motif |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |**C** |**G** |**G** |**C** |**A** |**G** |  |  |


- Y a t-il correspondance ? -> OUI
  - On décale à gauche
  - Y a t-il correspondance ? -> OUI
    - On décale à gauche
    - ...
    
**Le motif est donc présent dans notre chaîne.**

<video controls src="Boyer-Moore.mp4" style="border:1px solid black">
    
Exemple de cette vidéo : texte = ATAACAGGAGTAAATAACGGCTGGAGTAAAT et motif = CGGCTC

## Principe synthétique

- Prétaitement du motif : réalisation de la table de sauts
- On bouche jusqu'à la fin de la chaine
  - On boucle sur le motif en partant de la fin tant qu'il y a correspondance
    - Si on arrive à la fin du motif, le motif existe
    - Sinon, on regarde dans la table des sauts si la lettre est présente
      - Si la lettre est présente on effectue le saut correspondant **- le décalage effectué** si celui est positif, sinon 1
      - Sinon, on effectue le saut maximal
 
Voir https://en.wikipedia.org/wiki/Boyer%E2%80%93Moore%E2%80%93Horspool_algorithm

### Prétraitement du motif

Le but est d'obtenir l'écart minimal entre une lettre du motif et la fin du motif (la dernière lettre est omise)
Pourquoi minimal ? car la même lettre peut apparaitre plusieurs fois.

Ainsi pour notre exemple:'CGGCAG' on doit obtenir {'C':2, 'G':3, 'A':1}

Pour "rappel" on doit obtenir {'r':5,'a':4,'p':2,'e:1}

**Implémentation en python**

```python
def shift(pattern):
    """
    Retourne le dictionnaire des sauts pour les différentes lettres du motif (on exclu la dernière lettre)"""
    pass

```

Ajouter ces tests dans le fichier *test_horspool.py*:

```python
#   ------SHIFT----------------------
PATTERN = "aBcdCba"
DATAS_SHIFT = (
    # PATTERN         CASE_SENSITIVE  EXPECTED
    (PATTERN,         True,           {'a': 6, 'B': 5, 'c': 4, 'd': 3, 'C': 2, 'b': 1}),
    (PATTERN.lower(), True,           {'a': 6, 'b': 1, 'c': 2, 'd': 3}),
    (PATTERN.title(), True,           {'A': 6, 'b': 1, 'c': 2, 'd': 3}),
    (PATTERN,         False,          {'A': 6, 'B': 1, 'C': 2, 'D': 3}),
)
for pattern, case_sensitive, expected in DATAS_SHIFT: 
    result = shift(pattern, case_sensitive=case_sensitive)
    assert expected==result, f"Le motif '{pattern}' (case_sensitive={case_sensitive}) devrait donner {expected}```

<div class="alert alert-info">
    L'implémentation de l'algorithme de Horspool est quasi identique à celle de l'algorithme naïf sauf que l'on décale selon la table des sauts si le caractère est présent, de la longueur du motif sinon et que l'on parcourt le motif de droite à gauche.
</div>

```python
def search(text, pattern, case_sensitive=True):
    t = len(text)    # Longueur du texte
    p = len(pattern) # Longueur du motif
    if t == 0 or p == 0 or p>t:# Si le texte ou le motif est vide ou +grand que le texte -> liste vide
        return []
    # Construction de la table de décalage
    # on intialise l'index i à 0
    # on intialise les résultats à une liste vide.
    # tant qu'on n'est pas à la fin du texte
        # on initialise j à la valeur d'index de la derniere lettre du motif
        # si on est en case_sensitive:
            # On recule dans le motif tant que les caractères correspondent et que on n'a pas atteint le début du motif
                #on décrémente j
        # Sinon
            # On recule dans le motif tant que les caractères correspondent et que on n'a pas atteint le début du motif
                #on décrémente j
        # Si j vaut -1
            # on ajoute i au résultat
            # on ajoute 1 à i
        # Sinon
            # si on est en case_sensitive:
                # on récupère le saut
            # sinon
                # on récupère le saut
            decalages_effectues = (p-1)-j
            # le saut effectif vaut le saut - le décalage effectué
            # si le saut effectif est négatif:
                # on le passe à 1 (cas foireux)
            # on incrémente i de saut effectif
    #retourner le décalage

```

Ajouter ces test à votre fichier *test_horspool.py*.
```python
#   --------SEARCH-------------
for text, pattern, case_sensitive, expected in DATAS_SEARCH:
    result = search(text, pattern, case_sensitive=case_sensitive)
    assert expected==result, f"La recherche de '{pattern}' dans '{text}' aurait du être {expected} mais c'est {result}."
```

## Coût de cet algorithme

Ici le coût est beaucoup plus complexe à définir mais dans notre exemple il est évident qu'il est bien moindre que l'algorithme naïf.

Qui plus est, plus le motif est long plus l'algorithme est efficace.

## Exemple concret

Le *meilleur des mondes* de l'auteur *H. G. Wells* est disponible ici: https://www.gutenberg.org/cache/epub/60656/pg60656.txt

- Ecrire un programme qui télécharge le contenu de cette url et l'écrit dans un fichier *le_meilleur_des_mondes.txt*.
- A l'aide de votre fonction *search*
  - Comptez le nombre d'occurrences de 
    - 'invasion'
    - 'compositeur'
    - ' nsi '
  - Trouver l'emplacement des occurrences citées préalablement.
- Comparer le temps de traitement pour ne pas trouver l'occurrence * nsi * avec votre programme et avec l'algorithme naïf.

Voici un fichier qui utilise le module *timeit* et qui permet de comparer l'algo naïf et celui de Horspool.

```python
import timeit

SETUP_CODE = """
from horspool import naive_search, search, shift
roman_file = open("le_meilleur_des_mondes.txt","r",encoding='utf8')
roman = roman_file.read()
data = roman.replace('\\n', ' ')
pattern = "Personne n’aurait cru, dans les dernières années du dix-neuvième siècle,"
"""

TEST_NAIVE_CODE = """
naive_search(data, pattern, case_sensitive=False)"""

TEST_HORSPOOL_CODE = """
search(data, pattern, case_sensitive=False)"""

number = 10
naive_delay = timeit.timeit(setup=SETUP_CODE,
                          stmt=TEST_NAIVE_CODE,
                          number=number)


horspool_delay = timeit.timeit(setup=SETUP_CODE,
                          stmt=TEST_HORSPOOL_CODE,
                          number=number)

print(f'Naive search time: {naive_delay}')
print(f'Horspool search time: {horspool_delay}')
```

Faire des essais et conclure.

## webographie

- [Eduscol](https://cache.media.eduscol.education.fr/file/NSI/63/5/RA20_NSI_G_T_boyer-moore_1298635.pdf)
- [Algorithme de Boyer-Moore : Wikipedia](https://fr.wikipedia.org/wiki/Algorithme_de_Boyer-Moore)
- [Algorithme de Boyer-Moore-Horspool : Wikipedia](https://fr.wikipedia.org/wiki/Algorithme_de_Boyer-Moore-Horspool)


[Accueil](../../../index.ipynb) > [Sommaire de Terminale](../../index.ipynb)